# 9.1 Agent 核心概念演示

本笔记本将演示 AI Agent 的基本概念和工作流程。

**预估成本**: ~$0.02 (使用 GPT-4o-mini)

**安装依赖**:
```bash
pip install openai
```

In [ ]:
# 安装依赖
!pip install openai -q

In [ ]:
import os
from openai import OpenAI

# 初始化客户端
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# 检查 API key
if not os.environ.get("OPENAI_API_KEY"):
    print("⚠️  警告: 未设置 OPENAI_API_KEY 环境变量")
    print("请运行: export OPENAI_API_KEY='your-api-key'")
else:
    print("✓ API key 已设置")

## Demo 1: 简单 LLM 调用 vs Agent

对比简单的 LLM 调用和 Agent 的行为差异。

In [ ]:
# 简单 LLM 调用
def simple_llm_call(question: str) -> str:
    """
    简单的 LLM 调用，一次性返回答案
    """
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": question}],
    )
    return response.choices[0].message.content

# 测试问题
question = "北京现在几点？那里现在是白天还是晚上？"

print("=" * 80)
print("简单 LLM 调用")
print("=" * 80)
print(f"问题: {question}\n")

answer = simple_llm_call(question)
print(f"回答: {answer}\n")

print("观察:")
print("- LLM 直接给出答案（可能是猜的或基于训练数据）")
print("- 没有调用任何工具获取实时信息")
print("- 答案可能不准确")

## Demo 2: Agent 的循环推理过程

演示 Agent 的 Perceive → Plan → Act → Observe 循环。

In [ ]:
import time
from datetime import datetime

def simulate_agent_loop(task: str, max_steps: int = 5):
    """
    模拟 Agent 的推理循环
    """
    print("=" * 80)
    print("Agent 推理循环演示")
    print("=" * 80)
    print(f"任务: {task}\n")
    
    # 模拟的工具
    def get_current_time(location: str) -> str:
        # 简化：直接返回系统时间
        return datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Agent 循环
    print("[Step 1] Perceive (感知)")
    print(f"  理解任务: 需要获取北京当前时间并判断是白天还是晚上")
    time.sleep(1)
    
    print("\n[Step 2] Plan (规划)")
    print(f"  制定计划:")
    print(f"    1. 调用时间工具获取北京当前时间")
    print(f"    2. 根据时间判断是白天还是晚上")
    print(f"    3. 返回结果")
    time.sleep(1)
    
    print("\n[Step 3] Act (执行)")
    print(f"  调用工具: get_current_time('Beijing')")
    current_time = get_current_time("Beijing")
    time.sleep(1)
    
    print("\n[Step 4] Observe (观察)")
    print(f"  工具返回: {current_time}")
    print(f"  分析: 提取小时数并判断")
    time.sleep(1)
    
    # 判断白天/晚上
    hour = datetime.now().hour
    is_daytime = 6 <= hour < 18
    
    print("\n[Step 5] Act (再次执行)")
    print(f"  生成最终答案")
    time.sleep(1)
    
    print("\n" + "=" * 80)
    print("最终答案")
    print("=" * 80)
    result = f"北京当前时间是 {current_time}，现在是{'白天' if is_daytime else '晚上'}"
    print(result)
    
    print("\n观察:")
    print("- Agent 经过多个步骤的推理")
    print("- 主动调用工具获取实时信息")
    print("- 基于工具返回的结果进行推理")
    print("- 答案准确且有依据")

# 运行模拟
simulate_agent_loop("北京现在几点？那里现在是白天还是晚上？")

## Demo 3: Agent 的自主决策能力

演示 Agent 如何自主决定调用哪些工具、调用几次。

In [ ]:
def demonstrate_autonomous_decision():
    """
    演示 Agent 的自主决策
    """
    print("=" * 80)
    print("Agent 自主决策演示")
    print("=" * 80)
    
    # 不同复杂度的任务
    tasks = [
        {
            "task": "什么是 Python?",
            "complexity": "简单",
            "tools_needed": [],
            "steps": 1,
            "reason": "直接回答即可，不需要工具",
        },
        {
            "task": "计算 123 + 456",
            "complexity": "简单",
            "tools_needed": ["calculator"],
            "steps": 2,
            "reason": "需要调用计算器工具",
        },
        {
            "task": "Python 的创始人是谁？他出生在哪里？",
            "complexity": "中等",
            "tools_needed": ["search", "search"],
            "steps": 4,
            "reason": "需要搜索两次：先找创始人，再找出生地",
        },
        {
            "task": "分析最近一个月的销售数据趋势",
            "complexity": "复杂",
            "tools_needed": ["query_database", "analyze_data", "plot_chart", "generate_report"],
            "steps": 8,
            "reason": "需要多个工具协作，步骤不确定",
        },
    ]
    
    for i, task_info in enumerate(tasks, 1):
        print(f"\n任务 {i}: {task_info['task']}")
        print(f"复杂度: {task_info['complexity']}")
        print(f"需要的工具: {task_info['tools_needed'] if task_info['tools_needed'] else '无'}")
        print(f"预估步骤: {task_info['steps']}")
        print(f"原因: {task_info['reason']}")
        print("-" * 80)
    
    print("\n关键洞察:")
    print("1. Agent 根据任务复杂度自主决定是否使用工具")
    print("2. Agent 可以决定调用多少次工具")
    print("3. Agent 可以组合多个工具完成复杂任务")
    print("4. 人类不需要编写 if-else 逻辑")

demonstrate_autonomous_decision()

## Demo 4: 对比三种模式

可视化对比简单聊天机器人、Function Calling、Agent 的区别。

In [ ]:
def compare_three_patterns():
    """
    对比三种模式
    """
    print("=" * 80)
    print("三种模式对比")
    print("=" * 80)
    
    print("\n任务: '帮我查一下北京的天气，如果下雨就推荐带伞'\n")
    
    # 模式 1: 简单聊天机器人
    print("【模式 1】简单聊天机器人")
    print("-" * 80)
    print("流程:")
    print("  User → LLM → Response")
    print("\n行为:")
    print("  LLM: '根据我的训练数据，北京这个季节可能会下雨，建议带伞'")
    print("\n问题:")
    print("  ❌ 没有实时数据")
    print("  ❌ 答案可能不准确")
    print("  ❌ 无法执行操作\n")
    
    # 模式 2: Function Calling
    print("【模式 2】Function Calling")
    print("-" * 80)
    print("流程:")
    print("  User → LLM → Function Call → Tool → Result → LLM → Response")
    print("\n行为:")
    print("  # 你需要写代码决定何时调用工具")
    print("  if '天气' in user_message:")
    print("      weather = call_weather_api('Beijing')")
    print("      if weather['rain']:")
    print("          return '北京有雨，记得带伞'")
    print("\n优势:")
    print("  ✓ 有实时数据")
    print("  ✓ 答案准确")
    print("\n问题:")
    print("  ❌ 需要人工编写调用逻辑")
    print("  ❌ 无法处理复杂场景\n")
    
    # 模式 3: AI Agent
    print("【模式 3】AI Agent")
    print("-" * 80)
    print("流程:")
    print("  User → Agent → [Think → Act → Observe] × N → Response")
    print("\n行为:")
    print("  Step 1:")
    print("    Think: '我需要先查询北京的天气'")
    print("    Act: call_weather_api('Beijing')")
    print("    Observe: {'temp': 15, 'condition': 'rainy'}")
    print("  Step 2:")
    print("    Think: '天气显示有雨，我应该推荐带伞'")
    print("    Act: generate_response()")
    print("    Result: '北京今天有雨，气温15度，建议带伞'")
    print("\n优势:")
    print("  ✓ 自主决策何时调用工具")
    print("  ✓ 可以调用多个工具")
    print("  ✓ 根据结果动态调整")
    print("  ✓ 处理复杂多步任务\n")
    
    print("=" * 80)
    print("总结")
    print("=" * 80)
    print("简单任务 → 简单聊天机器人")
    print("固定流程 → Function Calling")
    print("复杂任务 → AI Agent")

compare_three_patterns()

## 总结

通过本笔记本，你应该理解了:

### 1. Agent 的核心特征
- **自主性 (Autonomy)**: AI 自己决定做什么
- **循环推理 (Loop)**: Perceive → Plan → Act → Observe
- **工具使用 (Tool Use)**: 主动调用工具获取信息

### 2. Agent vs 其他模式
- **简单 LLM**: 一次性对话，无工具
- **Function Calling**: 可以用工具，但需要人工编排
- **Agent**: 自主决策，循环执行

### 3. Agent 适用场景
✅ **适合**:
- 多步推理任务
- 不确定性任务
- 需要工具组合
- 需要中间判断

❌ **不适合**:
- 简单问答
- 固定流程
- 实时性要求高
- 成本敏感

### 下一步

学习 9.2 ReAct 模式，手写一个真正的 Agent！